In [1]:
from collections import defaultdict
from itertools import product
from more_itertools import with_iter
from networkx import Graph
from networkx import bfs_tree
from networkx import all_shortest_paths

# Part 1

In [2]:
lines = [l.strip() for l in with_iter(open("input.txt"))]
# lines = [l.strip() for l in with_iter(open("test4.txt"))]

class Actor(object):
    def __init__(self, name, strength):
        self.name = name
        self.hp = 200
        self.strength = strength

class PlayingField(object):
    def __init__(self, lines, strength):
        self.lines = lines
        self.height = len(lines) 
        self.width = len(lines[0]) 
        self.lines_cleared = [l.replace("G", ".").replace("E", ".") for l in self.lines]
        self.graph = Graph()
        self.e_count = 0
        self.g_count = 0
        self.elf_died = False
        
        node_counter = 0
        for y, line in enumerate(self.lines):
            for x, c in enumerate(line):
                if c != '#': 
                    if c in ("E", "G"):
                        if c == "E": 
                            actor = Actor("E", strength)
                            self.e_count += 1
                        else:
                            actor = Actor("G", 3)
                            self.g_count += 1
                        self.graph.add_node((y,x), actor=actor)
                    else:
                        self.graph.add_node((y,x), actor=None)
                        
        for (y,x) in product(range(self.height), range(self.width)):
            if (y,x) in self.graph.nodes:
                for nei in [(y-1,x), (y,x-1), (y,x+1), (y+1,x)]:
                    if nei in self.graph.nodes:
                        self.graph.add_edge(nei,(y,x))
                        
    def is_running(self):
        return self.g_count > 0 and self.e_count > 0
        
    def turn(self):
        if not self.is_running():
            return False
        
        actors = []
        for n in self.graph.nodes():
            a = self.graph.node[n]["actor"]
            if a is not None:
                actors.append(n)
        
        for n in actors:
            self.resolve(n)
            if not self.is_running():
                return False
        return True
        
    def resolve(self, n):
        if self.graph.node[n]["actor"] is None: # dead
            return
        if self.should_move(n):
            n = self.move(n)
        if n is not None and self.should_attack(n):
            self.attack(n)
            
    def should_attack(self, n):
        if not self.is_running():
            return False
        return not self.should_move(n)
        
    def should_move(self, n):
        if not self.is_running():
            return False
        a = self.graph.node[n]["actor"]
        for nei in self.graph.neighbors(n):
            an = self.graph.node[nei]["actor"]
            if an is not None and an.name != a.name and an.hp > 0:
                return False
        return True

    def move(self, n):
        name = self.graph.node[n]["actor"].name
        effective = self.effective_graph(n, name)
        bfs = bfs_tree(effective, n)
        found = None
        for nei in bfs:
            a = self.graph.node[nei]["actor"]
            if a is not None and a.name != name:
                found = nei
                break
        else:
            return
        
        node_to = self.shortest_path(effective, n, found)[1]
        
        k = self.graph.node[n]["actor"]
        self.graph.node[n]["actor"] = None
        self.graph.node[node_to]["actor"] = k

        return node_to

    def effective_graph(self, node, name):
        copy = self.graph.copy()
        for n in self.graph.nodes():
            a = self.graph.node[n]["actor"]
            if a is not None:
                if n == node:
                    continue
                elif a.name == name:
                    copy.remove_node(n)
        return copy
    
    def attack(self, n):
        a = self.graph.node[n]["actor"]
        candidates = []
        for nei in self.graph.neighbors(n):
            an = self.graph.node[nei]["actor"]
            if an is not None and an.name != a.name and an.hp > 0:
                y, x = nei
                candidates.append((an, y, x))
        if len(candidates) > 0:
            min_hp = min(c.hp for c, _, _ in candidates)
            candidates = [(c,y,x) for (c,y,x) in candidates if c.hp == min_hp]
            best, best_y, best_x = None, None, None
            for c, y, x in candidates:
                if best is None:
                    best, best_y, best_x = c, y, x
                elif y < best_y or (y == best_y and x < best_x):
                     best, best_y, best_x = c, y, x

            best.hp -= a.strength
            if best.hp <= 0:
                if best.name == "E": 
                    self.e_count -= 1
                    self.elf_died = True
                else: 
                    self.g_count -= 1
                    
                self.graph.node[(best_y, best_x)]["actor"] = None
    
    def shortest_path(self, effective, n, found):
        "Cut down shortest paths to the one that honors reading order"
        best = None
        for path in all_shortest_paths(effective, n, found):
            if best is None:
                best = path
                continue
            for (yp, xp), (yb, xb) in zip(path, best):
                if yp < yb:
                    best = path
                    break
        return best
    
    def sum_hp(self):
        first = None
        result = 0
        for n in self.graph.nodes():
            a = self.graph.node[n]["actor"]
            if a is not None:
                if first is None: 
                    first = a.name
                elif first != a.name:
                    raise Exception()
                result += a.hp
        return result
                
    def __repr__(self):
        r = self.lines_cleared.copy()
        hps = defaultdict(list)
        for n in self.graph.nodes():
            a = self.graph.node[n]["actor"]
            if a is not None:
                y, x = n
                hps[y].append(f"{a.name}({a.hp})")
                s = list(r[y])
                s[x] = a.name
                r[y] = "".join(s)
                
        res = [l+"   "+", ".join(hps[i]) for i, l in enumerate(r)]
        return "\n".join(res)

In [3]:
pf = PlayingField(lines, strength=3)
print(pf)
i = -1
while pf.turn():
    i += 1
print(i)
print(pf.sum_hp())
print(i*pf.sum_hp())
print(pf)

################################   
###########....#################   
###########......G##############   G(200)
############.G......############   G(200)
############........############   
########..G#.............#######   G(200)
#########.G.G................#.#   G(200), G(200)
######..#.......G..............#   G(200)
#######...G.....G.....#........#   G(200), G(200)
########..............E....##.##   E(200)
###....G##GG........G.....######   G(200), G(200), G(200), G(200)
###.###.##............##.#######   
###G##.....G..#####...#######..#   G(200), G(200)
##........#..#######..#######..#   
#...........#########.##.#....##   
#...........#########.......####   
#.......E...#########.##......##   E(200)
#G...G...#..#########.###......#   G(200), G(200)
#...G.......#########E#.#.....##   G(200), E(200)
#....#.....E.#######.......E..##   E(200), E(200)
###.###.......#####...#.E....###   E(200)
######................#.E....###   E(200)
#######G.#...#..##.####.......##   G(200)
####

# Part 2

In [4]:
for strength in range(30, 50):
    print(strength, end=": ")
    pf = PlayingField(lines, strength=strength)
    i = 1
    while pf.turn() and not pf.elf_died:
        i += 1
    if not pf.elf_died and pf.e_count > 0:
        print("YES", end=" - ")
        print(i, end=", ")
        print(pf.sum_hp(), end=", ")
        print(i*pf.sum_hp())
        print(pf)
        break
    else:
        print("NO")

30: NO
31: NO
32: NO
33: NO
34: YES - 28, 1478, 41384
################################   
###########....#################   
###########.......##############   
############........############   
############........############   
########...#.............#######   
#########....................#.#   
######..#......................#   
#######...............#........#   
########............E.E....##.##   E(17), E(200)
###.....##..........E.....######   E(125)
###.###.##.........E..##.#######   E(200)
###.##EEE.....#####...#######..#   E(176), E(164), E(185)
##.....E..#..#######..#######..#   E(11)
#...........#########.##.#....##   
#.........E.#########.......####   E(200)
#...........#########.##......##   
#........#..#########.###......#   
#...........#########.#.#.....##   
#....#.......#######.....E....##   E(200)
###.###.......#####...#......###   
######................#......###   
#######..#...#..##.####.......##   
#######............######...####   
#######.##........